In [9]:
# default_exp json_file

In [10]:
# export

import logging
from dataclasses import dataclass
from typing import *
from pathlib import Path
import json

from cocorepr.utils import sort_dict
from cocorepr.coco import *

logger = logging.getLogger()

In [11]:
# hide
logging.getLogger().setLevel(logging.DEBUG)

In [12]:
#export

def load_json_file(annotations_json: Union[str, Path], *, kind: str = "object_detection") -> CocoDataset:
    from_dict_function = get_dataset_class(kind).from_dict

    annotations_json = Path(annotations_json)
    logger.info(f"Loading json file from file: {annotations_json}")
    ext = annotations_json.suffix
    if ext != '.json':
        raise ValueError(f'Expect .json file as input, got: {annotations_json}')

    D = json.loads(annotations_json.read_text())
    coco = from_dict_function(D)
    logger.info(f"Loaded from json file: {coco.to_full_str()}")
    return coco

In [13]:
# hide
PATH = '../examples/coco_chunk/json_file/instances_train2017_chunk3x2.json'
d = load_json_file(PATH)
  
display(d.info)
display(d.annotations[0])
display(d.images[0])
display(d.categories[0])

assert isinstance(d.info, CocoInfo)
assert isinstance(d.annotations[0], CocoObjectDetectionAnnotation)
assert isinstance(d.images[0], CocoImage)
assert isinstance(d.categories[0], CocoObjectDetectionCategory)

CocoInfo(year=2017, version='1.0', description='COCO 2017 Dataset', contributor='COCO Consortium', url='http://cocodataset.org', date_created='2017/09/01')

CocoObjectDetectionAnnotation(id=124710, image_id=140006, category_id=2, bbox=(496.52, 125.94, 143.48, 113.54), supercategory=None, area=9387.706200000002, iscrowd=0)

CocoImage(id=49428, coco_url='http://images.cocodataset.org/train2017/000000049428.jpg', width=640, height=427, license=6, file_name='000000049428.jpg', flickr_url='http://farm7.staticflickr.com/6014/5923365195_bee5603371_z.jpg', date_captured='2013-11-15 04:30:29')

CocoObjectDetectionCategory(id=1, name='person', supercategory='person')

In [15]:
# export

def dump_json_file(
    coco: CocoDataset,
    annotations_json: Union[str, Path],
    *,
    kind: str = "object_detection",
    skip_nulls: bool = False,
    overwrite: bool = False,
    indent: Optional[int] = 4,
) -> None:
    dataset_class = get_dataset_class(kind)
    if skip_nulls:
        to_dict_function = dataset_class.to_dict_skip_nulls
    else:
        to_dict_function = dataset_class.to_dict

    annotations_json = Path(annotations_json)
    logger.info(f"Dumping json file to file: {annotations_json}")
    if annotations_json.is_file() and not overwrite:
        raise ValueError(f"Destination json file already exists: {annotations_json}")
    raw = sort_dict(to_dict_function(coco))
    logger.info(f"Writing dataset {coco.to_full_str()} to json file: {annotations_json}")
    annotations_json.parent.mkdir(parents=True, exist_ok=True)
    annotations_json.write_text(json.dumps(raw, indent=indent))

In [16]:
# hide
import tempfile
tmp = tempfile.mktemp()

dump_json_file(d, tmp)
! cat {tmp} | jq .info
! cat {tmp} | jq .images[0]
! cat {tmp} | jq .annotations[0]
! cat {tmp} | jq .categories[0]

{
  "year": 2017,
  "version": "1.0",
  "description": "COCO 2017 Dataset",
  "contributor": "COCO Consortium",
  "url": "http://cocodataset.org",
  "date_created": "2017/09/01"
}
{
  "id": 49428,
  "coco_url": "http://images.cocodataset.org/train2017/000000049428.jpg",
  "width": 640,
  "height": 427,
  "license": 6,
  "file_name": "000000049428.jpg",
  "flickr_url": "http://farm7.staticflickr.com/6014/5923365195_bee5603371_z.jpg",
  "date_captured": "2013-11-15 04:30:29"
}
{
  "id": 124710,
  "image_id": 140006,
  "category_id": 2,
  "bbox": [
    496.52,
    125.94,
    143.48,
    113.54
  ],
  "supercategory": null,
  "area": 9387.706200000002,
  "iscrowd": 0
}
{
  "id": 1,
  "name": "person",
  "supercategory": "person"
}
